In [1]:
library(rvest)
library(stringr)
library(dplyr)
library(tidyr)
library(purrr)

Warning message:
“package ‘rvest’ was built under R version 4.1.2”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘tidyr’ was built under R version 4.1.2”


In [20]:
google_scholar_count <- function(query) {
  encoded_query <- URLencode(query)
  url <- paste0("https://scholar.google.com/scholar?q=", encoded_query)
  user_agent <- "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
  
  tryCatch({
    page <- read_html(url, user_agent = user_agent)
    stats <- page %>% 
      html_nodes("#gs_ab_md") %>% 
      html_text()
    
    count <- str_extract(stats, "About (\\d+(?:,\\d+)*) results") %>%
      str_replace("About ", "") %>%
      str_replace(" results", "") %>%
      str_replace_all(",", "") %>%
      as.numeric()
    
    return(count)
  }, error = function(e) {
    message("Error occurred: ", e$message)
    return(NA)
  })
}
search_genes <- function(gene_list, additional_terms) {
  results <- map_dfr(gene_list, function(gene) {
    query <- paste0('"', gene, '" ', additional_terms)
    count <- google_scholar_count(query)
    message(paste("Processed", gene, ":", count, "results"))
    Sys.sleep(900)  # Delay to avoid overwhelming the server
    tibble(Gene = gene, Count = count)
  })
  return(results)
}


In [22]:
# List of genes
genes <- c('WNT5A', 'INS')  # Add more genes as needed

# Additional search terms
additional_terms <- "gene stem cell differentiation beta cells Pancreatic β-Cells"

# Perform the search
results <- search_genes(genes, additional_terms)

Processed WNT5A : 828 results

Processed INS : 8830 results



In [10]:
genes <- c('MAFA', 'MAFB', 'MAFF', 'MAFG')  # Add more genes as needed
results <- search_genes(genes, additional_terms)


Processed MAFA : 4170 results

Processed MAFB : 1510 results

Processed MAFF : 147 results

Processed MAFG : 154 results



In [ ]:
results

Gene,Count
<chr>,<dbl>
MAFA,4170
MAFB,1510
MAFF,147
MAFG,154
